In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor
import threading
import csv
import re

# Base URL of the website
base_url = "https://cuir.car.chula.ac.th"

# Set headers to mimic a browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}


In [8]:
def collection_home_page( collection_link , last_offset ) :
    # Initialize the starting value
    offset = 0
    Thesis_2013 = 0
    Thesis_2014 = 0
    Thesis_2015 = 0
    Thesis_2016 = 0
    Thesis_2017 = 0
    Thesis_2018 = 0
    Thesis_2019 = 0
    Thesis_2020 = 0
    Thesis_2021 = 0
    Thesis_2022 = 0
    Thesis_2023 = 0
    
    lock = threading.Lock()


    # Base URL template
    url_template = '{collection_link}?offset={offset}'

    # Function to process each URL
    def process_page(offset):
        nonlocal Thesis_2013, Thesis_2014, Thesis_2015, Thesis_2016, Thesis_2017, Thesis_2018, Thesis_2019, Thesis_2020, Thesis_2021, Thesis_2022, Thesis_2023  # Ensure we modify the outer variables
        url = f'{collection_link}?offset={offset}'

        # try:
            # Make the HTTP GET request
        response = requests.get(url, headers=headers, timeout=5)
        response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx

        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(response.text, "lxml")

        # Extract year data and links using the appropriate selector
        years = soup.find_all('td', headers='t1')  

        if years:
            # Iterate over the years
            for year in years:
                year_text = year.text.strip()
                # Collect counts for the relevant years
                with lock:  # Acquire the lock to safely modify shared variables
                    if year_text == '2013' or year_text == '2556':
                        Thesis_2013 += 1
                    elif year_text == '2014' or year_text == '2557':
                        Thesis_2014 += 1
                    elif year_text == '2015' or year_text == '2558':
                        Thesis_2015 += 1
                    elif year_text == '2016' or year_text == '2559':
                        Thesis_2016 += 1
                    elif year_text == '2017' or year_text == '2560':
                        Thesis_2017 += 1
                    elif year_text == '2018' or year_text == '2561':
                        Thesis_2018 += 1
                    elif year_text == '2019' or year_text == '2562':
                        Thesis_2019 += 1
                    elif year_text == '2020' or year_text == '2563':
                        Thesis_2020 += 1
                    elif year_text == '2021' or year_text == '2564':
                        Thesis_2021 += 1
                    elif year_text == '2022' or year_text == '2565':
                        Thesis_2022 += 1
                    elif year_text == '2023' or year_text == '2566':
                        Thesis_2023 += 1
                # print(f"Finished offset: {offset}")

            # else:
            #     print("No year or link data found on the current page.")

        # except requests.exceptions.RequestException as e:
            # print(f"Error fetching URL at offset {offset}: {e}")

    # Number of worker threads to use
    max_workers = 10

    # Use ThreadPoolExecutor to run the process_page function concurrently
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Main batch processing
        while offset + 20 * max_workers <= last_offset:
            # Submit tasks to the executor for the current batch of offsets
            futures = [executor.submit(process_page, offset + i * 20) for i in range(max_workers)]
            offset += 20 * max_workers  # Increment offset for the next batch of requests

            # Wait for all futures in the current batch to finish
            for future in futures:
                future.result()

        # print(" ------------------------- LEFT -------------------------")

        # Handle the remaining offsets (less than one batch)
        while offset < last_offset:
            futures = [executor.submit(process_page, offset)]
            offset += 20  # Increment offset for each remaining page

            # Wait for the single future to finish
            for future in futures:
                future.result()

    # After the loop, print the count of thesis by year
    # print(falculty_name)
    # # print(offset)
    # print(f"Thesis in 2015: {Thesis_2015}")
    # print(f"Thesis in 2016: {Thesis_2016}")
    # print(f"Thesis in 2017: {Thesis_2017}")
    
    return( Thesis_2013 , Thesis_2014 , Thesis_2015 , Thesis_2016 , Thesis_2017 , Thesis_2018 , Thesis_2019 , Thesis_2020 , Thesis_2021 , Thesis_2022 , Thesis_2023 )

In [4]:
def falculty_page( falculty_link ) :
    
    # Make the request to fetch the page
    response = requests.get(falculty_link, headers=headers)
    response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, "lxml")

    # Find the <h4> tag with class "list-group-item-heading"
    collections = soup.find_all('h4', class_='list-group-item-heading')
    links = soup.find_all('h4', class_='list-group-item-heading')
    
    all_thesis_2013 = 0
    all_thesis_2014 = 0
    all_thesis_2015 = 0
    all_thesis_2016 = 0
    all_thesis_2017 = 0
    all_thesis_2018 = 0
    all_thesis_2019 = 0
    all_thesis_2020 = 0
    all_thesis_2021 = 0
    all_thesis_2022 = 0
    all_thesis_2023 = 0

    # Use regular expression to find the number in square brackets
    for collection , link in zip(collections , links) :
        last_offset_uncleaned = re.search(r'\[(\d+)\]', collection.text.strip())


        if last_offset_uncleaned:
            # If a match is found, print the number inside the brackets
            last_offset = int(last_offset_uncleaned.group(1))
            # print(last_offset)
        
            
        if last_offset > 0: 
            anchor = link.find('a', href=True)
            collection_link = urljoin(base_url, anchor['href'])
            t13 , t14 , t15 , t16 , t17 , t18 , t19 , t20, t21 , t22 , t23= collection_home_page( collection_link , last_offset)
            all_thesis_2013 += t13
            all_thesis_2014 += t14
            all_thesis_2015 += t15
            all_thesis_2016 += t16
            all_thesis_2017 += t17
            all_thesis_2018 += t18
            all_thesis_2019 += t19
            all_thesis_2020 += t20
            all_thesis_2021 += t21
            all_thesis_2022 += t22
            all_thesis_2023 += t23

    return  all_thesis_2013 , all_thesis_2014 , all_thesis_2015 , all_thesis_2016 , all_thesis_2017 , all_thesis_2018 , all_thesis_2019 , all_thesis_2020 , all_thesis_2021 , all_thesis_2022 , all_thesis_2023
            # print(last_offset)
        # else:
        #     # print(" No Data " )
    

    
    # return falculty_name , falculty_link
    

In [9]:
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# URL of the main page
url = 'https://cuir.car.chula.ac.th/handle/123456789/4587'  # Replace with the actual URL

# Make the request to fetch the page
response = requests.get(url)
response.raise_for_status()  # Raise an error for HTTP codes 4xx/5xx

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, "lxml")

# Find all the faculty descriptions and links
facs = soup.find_all('p', class_='collectionDescription')
links = soup.find_all('h4', class_='list-group-item-heading')

# Prepare CSV structure
output_file = "thesis_summary.csv"
# columns = ["Year"]
results_dict = {"2013": [], "2014": [], "2015": [], "2016": [], "2017": [], "2018": [], "2019": [], "2020": [], "2021": [], "2022": [], "2023": []}

# Transposed output file
transposed_file = "transposed_thesis_summary.csv"
transposed_rows = []  # To store the transposed rows

faculty = ['College of Population Studies',
            'College of Public Health Sciences',
            'Faculty of Allied Health Sciences',
            'Faculty of Architecture',
            'Faculty of Arts',
            'Faculty of Commerce and Accountancy',
            'Faculty of Communication Arts',
            'Faculty of Dentistry',
            'Faculty of Economics',
            'Faculty of Education',
            'Faculty of Engineering',
            'Faculty of Fine and Applied Arts',
            'Faculty of Law',
            'Faculty of Medicine',
            'Faculty of Nursing',
            'Faculty of Pharmaceutical Science',
            'Faculty of Political Science',
            'Faculty of Psychology',
            'Faculty of Science',
            'Faculty of Sports Science',
            'Faculty of Veterinary Science',
            'Graduate School',
            'Office of Research Affairs',
            'Institute of Asian Studies',
            'The Institute of Biotechnology and Genetic Engineering',
            'Institute of Thai Studies',
            'Language Institute',
            'Office of Academic Resources',
            'The Petroleum and Petrochemical College',
            'Sasin Graduate Institute of Business Administration']
faculty_idx = 0
# Iterate over faculties and fetch results
for fac, link in zip(facs, links):
    anchor = link.find('a', href=True)
    full_link = urljoin(base_url, anchor['href'])
    
    # Call the faculty_page function
    # faculty_name = anchor.text.split(" - ")[0].strip()
    result = falculty_page(full_link)
    print(f"Faculty: {fac.text.strip()}, Result: {result}")
    
    # Prepare a row for transposed CSV (faculty name + thesis data)
    faculty_row = []
    faculty_row.append(faculty[faculty_idx])
    # Add faculty name to the front of the row
    faculty_row.extend(result[:12])  # Appends result[0] to result[11] (first 12 elements)

    
    faculty_idx += 1
    # Append the faculty row to the transposed rows
    transposed_rows.append(faculty_row)

# Write transposed data to CSV
with open(transposed_file, mode="w", newline="", encoding="utf-8") as outfile:
    writer = csv.writer(outfile)

    # Write the header (first column is faculty names and subsequent columns are years)
    writer.writerow(["Faculty Names", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"])

    # Write each transposed row (faculty + thesis data)
    writer.writerows(transposed_rows)

print(f"Transposed CSV file '{transposed_file}' created successfully!")


Faculty: วิทยาลัยประชากรศาสตร์, Result: (10, 9, 14, 6, 6, 10, 5, 2, 8, 8, 1)
Faculty: วิทยาลัยวิทยาศาสตร์สาธารณสุข, Result: (50, 40, 41, 47, 55, 35, 35, 29, 22, 11, 4)
Faculty: คณะสหเวชศาสตร์, Result: (11, 13, 29, 20, 19, 8, 23, 20, 7, 10, 4)
Faculty: คณะสถาปัตยกรรมศาสตร์, Result: (87, 87, 83, 114, 81, 76, 73, 77, 59, 60, 13)
Faculty: คณะอักษรศาสตร์, Result: (96, 95, 97, 89, 74, 48, 36, 60, 51, 51, 28)
Faculty: คณะพาณิชยศาสตร์และการบัญชี, Result: (40, 35, 47, 29, 28, 21, 60, 51, 78, 106, 50)
Faculty: คณะนิเทศศาสตร์, Result: (71, 57, 61, 33, 43, 28, 72, 73, 72, 70, 14)
Faculty: คณะทันตแพทยศาสตร์, Result: (26, 27, 29, 40, 38, 39, 42, 49, 31, 27, 4)
Faculty: คณะเศรษฐศาสตร์, Result: (39, 48, 33, 31, 32, 32, 63, 84, 56, 37, 7)
Faculty: คณะครุศาสตร์, Result: (216, 227, 267, 241, 198, 164, 158, 200, 191, 120, 40)
Faculty: คณะวิศวกรรมศาสตร์, Result: (452, 404, 409, 371, 297, 301, 316, 254, 238, 218, 72)
Faculty: คณะศิลปกรรมศาสตร์, Result: (42, 46, 47, 42, 57, 70, 61, 65, 57, 63, 12)
Faculty: ค